from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import sql
from pyspark.sql import SQLContext


:


conf = SparkConf().setMaster("local[20]").setAppName("hw")
#sc = SparkContext(conf = conf)
sc = SparkContext()
sqlContext = sql.SQLContext(sc)

In [1]:
df = sqlContext.read.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true").load("/data/MSA_8050_Spring_18/proj09/final_merged_data.csv")

In [2]:
li=['business_name',
 'neighborhood',
 'address','postal_code',
 'latitude',
 'longitude','is_open',
 'categories','review_date','user_name','yelping_since',
 'friends',
 'user_useful',
 'user_funny',
 'user_cool',
 'fans',
 'elite', 'compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos']

In [3]:
for i in li:
    df=df.drop(i)

In [ ]:
#df.columns

In [ ]:
# df.select(['bus_business_id','review_count1']).show()

In [4]:
df = df.withColumnRenamed("review_count1","bus_review_count")

import pyspark.sql.functions as f
df = df.withColumn('index', f.monotonically_increasing_id())




# sort ascending and take first 100 rows for df1
df1 = df.sort('index').limit(100000)

df=df1

In [5]:
df.columns

['bus_business_id',
 'city',
 'state',
 'stars',
 'bus_review_count',
 'review_id',
 'user_id',
 'business_id',
 'review_stars',
 'text',
 'review_useful',
 'review_funny',
 'review_cool',
 'user_user_id',
 'review_count',
 'average_stars']

df=df.drop('index')

In [ ]:
df.count()

In [ ]:
#df.columns

In [6]:
from pyspark.sql.functions import sum
from pyspark.sql.functions import col

In [7]:
df.groupBy('city').agg(sum('bus_review_count').alias('Reviews')).sort(col('Reviews').desc()).take(5)

[Row(city=u'Las Vegas', Reviews=6040013),
 Row(city=u'Phoenix', Reviews=709873),
 Row(city=u'Scottsdale', Reviews=341888),
 Row(city=u'Toronto', Reviews=301586),
 Row(city=u'Henderson', Reviews=259118)]

In [8]:
df_top_city=df.where(df.city.isin(['Phoenix','Las Vegas','Scottsdale','Toronto','Henderson']))

In [9]:
rdd_top_city=df_top_city.rdd.map(list)

In [10]:
from collections import OrderedDict
import string
import nltk
from nltk.corpus import stopwords

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kingale1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
words =  rdd_top_city.map(lambda x:(x[1], x[9].encode("utf-8").lower().translate(None,string.punctuation).strip()))


In [ ]:
words.take(2)

In [12]:
stopwrds = set(stopwords.words('english'))

In [13]:
X = words.reduceByKey(lambda x,y:x+y).map(lambda x:(x[0],x[1].split(" "))).map(lambda x:(x[0],{i:x[1].count(i) for i in set(x[1])}))


In [14]:
Z = X.map(lambda x:(x[0],OrderedDict(sorted(x[1].items(), key = lambda y:y[1], reverse = True))))


In [15]:
top_11 = Z.map(lambda x: (x[0], [i for i in x[1] if i not in stopwrds][:11]))


In [16]:
for k,v in top_11.collect():
    print '{0} : {1}'.format(k,list(v)[1:11])

Toronto : ['place', 'food', 'good', 'great', 'like', 'one', 'service', 'really', 'would', 'get']
Phoenix : ['place', 'food', 'good', 'great', 'like', 'get', 'time', 'service', 'one', 'would']
Scottsdale : ['great', 'place', 'food', 'good', 'service', 'like', 'time', 'would', 'get', 'one']
Las Vegas : ['good', 'food', 'place', 'great', 'like', 'get', 'time', 'service', 'one', 'would']
Henderson : ['food', 'good', 'like', 'great', 'place', 'time', 'get', 'really', 'one', 'service']
